# Magister en Ciencia de Datos - UDD
## DBAnalytics (Ciencia de Datos aplicada)
### Sprint 2: Datos transformados II

Realizar una búsqueda de hiper-parámetros para entrenar el modelo usando el algoritmo Gradient Boosting Trees.
Explorar el impacto de cada feature en el modelo, de dos maneras: 
Tests KS univariados comparando las poblaciones Churn vs No-Churn
Performance del modelo entrenado con y sin el feature (sensibilidad)
Calcular el uplift del resultado final para el segmento de los top 10K msno’s. (%churn rate top 10K vs %churn rate total)

In [1]:
import pandas as pd

* Lectura de archivos de entrenamiento y test

In [2]:
!bq ls DATASET

         tableId           Type    Labels   Time Partitioning   Clustered Fields  
 ------------------------ ------- -------- ------------------- ------------------ 
  members_v3               TABLE                                                  
  rfm_07                   TABLE                                                  
  rfm_14                   TABLE                                                  
  rfm_30                   TABLE                                                  
  rfm_60                   TABLE                                                  
  rfm_90                   TABLE                                                  
  sample_submission_v2     TABLE                                                  
  sample_submission_zero   TABLE                                                  
  train_sorted_v2          TABLE                                                  
  train_v2                 TABLE                                                  
  tr

In [3]:
!gsutil ls -a gs://kk_data_udd

gs://kk_data_udd/day_listen.csv#1564861559107461
gs://kk_data_udd/df_test.csv#1565240427109815
gs://kk_data_udd/df_train.csv#1565240406697715
gs://kk_data_udd/fea_rank.csv#1565661603594903
gs://kk_data_udd/members_v3.csv#1563566790239785
gs://kk_data_udd/sample_submission_v2.csv#1563580288727022
gs://kk_data_udd/sample_submission_zero.csv#1563580145138161
gs://kk_data_udd/sub_age_xgb_pred.csv#1565055727433942
gs://kk_data_udd/sub_day_listen.csv#1564861658307683
gs://kk_data_udd/sub_reg_via_xgb_pred.csv#1565055581900599
gs://kk_data_udd/sub_user_satisfaction.cvs#1564861608790636
gs://kk_data_udd/test_sorted_v1.csv#1565242819914404
gs://kk_data_udd/train.csv#1563565831541482
gs://kk_data_udd/train_sorted_v1.csv#1565243444686022
gs://kk_data_udd/train_sorted_v2.csv#1565583113260731
gs://kk_data_udd/train_v2.csv#1563580263806878
gs://kk_data_udd/transactions.csv#1563580088583483
gs://kk_data_udd/transactions_v2.csv#1563580288931202
gs://kk_data_udd/user_label_201702.csv#1563681052454642
gs

In [14]:
df = pd.read_csv("gs://kk_data_udd/train_sorted_v2.csv", nrows=20)

In [15]:
df.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,discount_120,discount_149,discount_180,discount_20,discount_30,discount_50,md_-1,md_0,day_listen,user_latent_satisfaction
0,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,1.0,9.0,2005.0,39,30,149,...,0,0,0,0,0,0,0,1,33.0,0.820694
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,1.0,9.0,2005.0,39,30,149,...,0,0,0,0,0,0,0,1,33.0,0.820694
2,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1,18.0,22.0,2.0,9.0,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,152.0,0.919394
3,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,0,18.0,22.0,2.0,9.0,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,152.0,0.919394
4,MkuWz0Nq6/Oq5fKqRddWL7oh2SLUSRe3/g+XmAWqW1Q=,1,11.0,31.0,2.0,9.0,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,4.0,1.000000


In [4]:
#!bq load --quote "" --source_format=CSV --skip_leading_rows=1 DATASET.train_sorted_v2 gs://kk_data_udd/train_sorted_v2.csv ./schema_sprint4.json

In [18]:
df = pd.read_csv("gs://kk_data_udd/test_sorted_v1.csv", nrows=20)

In [19]:
df.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,r_month,r_day,m_month,m_day,t_month,t_day,l_month,l_day,autorenew_&_not_cancel,notAutorenew_&_cancel
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,1.0,0.0,NaN,7.0,2015.0,41,30,99,...,7.0,18.0,4,18,3,18,1.0,15.0,1,1
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,4.0,28.0,1.0,9.0,2005.0,34,30,149,...,10.0,30.0,4,30,3,31,3.0,31.0,1,1
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,4.0,34.0,1.0,7.0,2014.0,41,30,99,...,11.0,1.0,4,15,3,15,3.0,28.0,1,1
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,NaN,NaN,NaN,NaN,NaN,41,30,99,...,NaN,NaN,4,27,3,27,NaN,NaN,1,1
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,1.0,0.0,NaN,13.0,2016.0,30,30,129,...,12.0,22.0,4,21,3,22,2.0,16.0,1,1


In [45]:
schema = "["
for column in df.columns:
    schema += "{" + "'name'" + ":'" + column + "','type':'" + str(df[column].dtype) + "'},"
schema

In [ ]:
#!bq load --quote "" --source_format=CSV --skip_leading_rows=1 DATASET.test_sorted_v1 gs://kk_data_udd/test_sorted_v1.csv ./schema_sprint4_test.json

In [49]:
%%bigquery
select * from DATASET.test_sorted_v1 limit 5

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,r_month,r_day,m_month,m_day,t_month,t_day,l_month,l_day,autorenew_y_not_cancel,notAutorenew_y_cancel
0,76HmcWH+ThBjAXMeyy3m3TSEAignH1+NqsJhGdKK0Ws=,0,NaN,NaN,NaN,NaN,NaN,30,30,149,...,NaN,NaN,4,23,3,24,None,None,1,1
1,RHXbnBkbSy/P8QLHZ6gY7uTWUz0+PARxBFRuXQsNU5Y=,0,13.0,23.0,1.0,9.0,2017.0,17,240,0,...,3.0,27.0,4,2,3,28,None,None,0,0
2,5Umo7ydygyTk+LvGDGfJa0l0+zQHzC5rMW+fzx6YGLQ=,0,NaN,NaN,NaN,NaN,NaN,30,30,149,...,NaN,NaN,4,2,3,3,None,None,1,1
3,uv3mCWik4lXK/ri4RsmC70SrJzFQ8WEkSph7yWcHFw0=,0,NaN,NaN,NaN,NaN,NaN,30,30,149,...,NaN,NaN,4,12,3,13,None,None,1,1
4,6NtCcXbBzNsQqs+AFtVeTDTUxkUcBmQ0FeTYJRsN69s=,0,NaN,NaN,NaN,NaN,NaN,30,30,149,...,NaN,NaN,4,3,3,4,None,None,1,1
